In [1]:
import cv2
import numpy as np
import copy

In [2]:
def showImg(img):
    if type(img) != list:
        cv2.imshow("window", img)

    else:
        for i, im in enumerate(img):
            cv2.imshow("window"+str(i), im)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def threshImg(img):   
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, imgThresh = cv2.threshold(imgGray, 100, 255, cv2.THRESH_BINARY_INV)
    
    return imgThresh

# define template

## apply perspective transform for consistency with input 
## find x,y locs of spots

In [4]:
templateImgPath = "v3.4_blank_filled_corrected0.jpg"

tempImg = cv2.imread(templateImgPath)

# apply rotations, resizing etc

imgResize = cv2.resize(tempImg, (0,0), fx= 0.5, fy=0.5, interpolation= cv2.INTER_AREA)
imgGray = cv2.cvtColor(imgResize, cv2.COLOR_BGR2GRAY)
imgBilat = cv2.bilateralFilter(imgGray, 11,500,0)
imgEdges = cv2.Canny(imgBilat, 20,100 )

conts = cv2.findContours(imgEdges, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

areas =[]
for c in conts[0]:
    areas.append([c, cv2.contourArea(c)])

#find top 10 biggest contours by area
sortedLengths = sorted(areas, key=lambda x:x[1], reverse=True)
topConts = sortedLengths[:10]

#find outer rectangle 
outerBoxCnt = None
for c in topConts:
    # approximate the contour
    peri = cv2.arcLength(c[0], True)
    #approximate curve to check if its rectangular
    approx = cv2.approxPolyDP(c[0], 0.015 * peri, True)
    if len(approx) == 4:
        outerBoxCnt = approx
        break

pts = outerBoxCnt.reshape(4,2)

#ordered from 0-3: top left, top right, bottom right, bottom left (go clockwise around rect)
orderedPts = np.zeros((4,2), dtype='float32')

#largest sum of x+y = bottom right
#smallest sum of x+y = top left
orderedPts[0] = pts[np.argmin(pts.sum(axis=1))]
orderedPts[2] = pts[np.argmax(pts.sum(axis=1))]

#smallest difference x-y = top right
orderedPts[1] = pts[np.argmin(np.diff(pts, axis=1))]
#largest difference x-y = bottom left
orderedPts[3] = pts[np.argmax(np.diff(pts, axis=1))]

#unpack ordered pts to find widths and heights
(tl, tr, br, bl) = orderedPts

#use euclidean distances (finally a use for pythagoras lol) - taken from pyimagesearch.com
widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))

#find max heights/widths - i.e from top/bottom, left/right
maxWidth = max(int(widthA), int(widthB))
maxHeight = max(int(heightA), int(heightB))

#initialise dst array for transformation
dst = np.array([
    [0, 0],
    [maxWidth-1, 0],
    [maxWidth -1, maxHeight-1 ],
    [0, maxHeight-1]], dtype = "float32")

#transformation matrix
matrix = cv2.getPerspectiveTransform(orderedPts, dst)

#transform image and resize to original size (map spots to correct locations)
tempImg = cv2.warpPerspective(imgResize, matrix, (maxWidth, maxHeight))

imgGray = cv2.cvtColor(tempImg, cv2.COLOR_BGR2GRAY)
ret, imgThresh = cv2.threshold(imgGray, 100, 255, cv2.THRESH_BINARY_INV)

imgBlank = np.full_like(tempImg, 255)

conts, hier = cv2.findContours(imgThresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

spotCentres=[]
widths =[]

for c in conts: 
    # compute enclosing rect to get center points
    x,y,w,h = cv2.boundingRect(c)
    ar = w/h
    if w<100 and h<100 and 0.8<ar<1.2:      
        #draw on blank to show correct detection        
        cv2.rectangle(tempImg,(x,y), (x+w, y+h), (0,0,255))        
        widths.append(w)
        xCentre = int(x+(w/2))
        yCentre = int(y+(h/2))
        spotCentres.append([xCentre,yCentre])
        #cv2.putText(tempImg, str(xCentre)+","+str(yCentre), (xCentre+10, yCentre+10), cv2.FONT_HERSHEY_PLAIN, 0.5, (0,0,0))

#sort by x, then y
sortedSpots = sorted(spotCentres, key=lambda x:(x[0], x[1]), reverse=False)
print(len(sortedSpots))

showImg(tempImg)

707


In [5]:
tempImgWidth= tempImg.shape[1]
tempImgHeight= tempImg.shape[0]

## initialise list of dictionaries with: questions: list of responses dictionary
    ## i.e. list of (questions:answer dictionaries)
                list of (answers:value, coord, filled) 

In [108]:
## access answers of question (use this to check filled status from image?)

def getQuestionDetails(questionString, templateList):
    '''return list of answer dictionaries for question (value, coord, filled) '''
    index = [ind for ind, qn in enumerate(templateList) if qn['question']==questionString][0]
    
    return templateList[index]['answers']

## fill in template

In [82]:
templateSpots=[]

### define generic function to generate questions for template

In [198]:
def generateQuestion(xRange, yRange, orient, questionName, answerValues, spotCoordsList):
    '''returns question dictionary 
    xRange/yRange = list of x and y coordinates
    orient = 'column' or 'row' or 'grid'
    questionName = str of question to associate with answerValues
    answerValues = list of values to associate with coordinates in xRange, 
    must be sorted from top to bottom (if orient = 'column'),
    left to right (if orient = 'row')
    left to right, then top to bottom - like reading english (if orient = 'grid')
    spotCoordsList = list of all coordinates from template to extract spots within xRange and yRange from.
    '''
    #return list of coordinates within range
    coordList = [i for i in spotCoordsList if xRange[0]<=i[0]<=xRange[1] and yRange[0]<=i[1]<=yRange[1]]
    
    #sort list by x if orient == 'row' or by y if column
    if orient =='row':
        coordList = sorted(coordList, key = lambda x:x[0])
    elif orient =='column':
        coordList = sorted(coordList, key = lambda x:x[1])
    elif orient=='grid':
        coordList = sorted(coordList,key =  lambda x:(x[0], x[1]))
    else:
        raise Exception("orient must be 'column' or 'row'")
    
    #map coordinates to values
    if len(answerValues) == len(coordList):
        valueCoords = zip(answerValues, coordList)
    else:
        raise Exception("answerValues length: ({}) must match coordList length: ({})".format(len(answerValues), len(coordList)))
    
    #add question to template
    questionDict = {'question': questionName, 'answers':[]}
    
    #add in answers
    for val, coord in valueCoords:
        questionDict['answers'].append({'value': val, 'coord':coord, 'filled': False})
    
    return questionDict

## helper function to visualise questions/values

In [202]:
def showQuestion(questionString, templateList, img, fontSize):
    for qn in getQuestionDetails(questionString, templateList):
        x,y = qn['coord']        
        cv2.circle(img, (x,y), 5, (0,0,255), -1)
        cv2.putText(img, str(qn['value']), (x+10, y), cv2.FONT_HERSHEY_SIMPLEX, fontSize, (0,0,0))
        
    showImg(img)

### define questions

In [89]:
templateSpots.append(generateQuestion([37,37], [321,371] , 'column', 'day1', [0,1,2,3], sortedSpots))

In [93]:
templateSpots.append(generateQuestion([60,65], [320,475], 'column', 'day2', [0,1,2,3,4,5,6,7,8,9], sortedSpots))

In [184]:
templateSpots.append(generateQuestion([90, 115], [320, 510], 'column', 'month', list(range(1,13)), sortedSpots))

In [192]:
templateSpots.append(generateQuestion([150, 160], [320, 390], 'column', 'year', list(range(2020,2025)), sortedSpots))

In [196]:
templateSpots.append(generateQuestion([200, 205], [465,490], 'column', 'shift', ['AM', 'PM'], sortedSpots))

In [200]:
templateSpots.append(generateQuestion([220,300], [465, 490], 'grid', 'cell', 
                 ['machined', 'bonded', 'isolast',
                  'flexVRings', 'custom', 'silastics', 'orings', 'cleanroom'], 
                 sortedSpots))

In [206]:
showQuestion('cell', templateSpots, oeeImg, 0.3)

## need to "manually" define questions, values : use slicing/sorting of spots

In [174]:
##use this to fill in templateSpots
pass

## perspective transform on input OEE form to "map" image to template

In [207]:
oeeImgPath = "oee_filled.jpg"

oeeImg = cv2.imread(oeeImgPath)

# apply rotations, resizing etc

imgResize = cv2.resize(oeeImg, (0,0), fx= 0.5, fy=0.5, interpolation= cv2.INTER_AREA)
imgGray = cv2.cvtColor(imgResize, cv2.COLOR_BGR2GRAY)
imgBilat = cv2.bilateralFilter(imgGray, 11,500,0)
imgEdges = cv2.Canny(imgBilat, 20,100 )

conts = cv2.findContours(imgEdges, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

areas =[]
for c in conts[0]:
    areas.append([c, cv2.contourArea(c)])

#find top 10 biggest contours by area
sortedLengths = sorted(areas, key=lambda x:x[1], reverse=True)
topConts = sortedLengths[:10]

#find outer rectangle 
outerBoxCnt = None
for c in topConts:
    # approximate the contour
    peri = cv2.arcLength(c[0], True)
    #approximate curve to check if its rectangular
    approx = cv2.approxPolyDP(c[0], 0.015 * peri, True)
    if len(approx) == 4:
        outerBoxCnt = approx
        break

pts = outerBoxCnt.reshape(4,2)

#order outer rectangle points from 0-3: top left, top right, bottom right, bottom left (go clockwise around rect)
orderedPts = np.zeros((4,2), dtype='float32')

#largest sum of x+y = bottom right
#smallest sum of x+y = top left
orderedPts[0] = pts[np.argmin(pts.sum(axis=1))]
orderedPts[2] = pts[np.argmax(pts.sum(axis=1))]

#smallest difference x-y = top right
orderedPts[1] = pts[np.argmin(np.diff(pts, axis=1))]
#largest difference x-y = bottom left
orderedPts[3] = pts[np.argmax(np.diff(pts, axis=1))]

#map onto template image width/height
maxWidth = tempImgWidth
maxHeight = tempImgHeight

#initialise dst array for transformation
dst = np.array([
    [0, 0],
    [maxWidth-1, 0],
    [maxWidth -1, maxHeight-1 ],
    [0, maxHeight-1]], dtype = "float32")

#transformation matrix
matrix = cv2.getPerspectiveTransform(orderedPts, dst)

#transform image and resize to original size (map spots to correct locations)
oeeImg = cv2.warpPerspective(imgResize, matrix, (maxWidth, maxHeight))

#showImg([oeeImg, tempImg])


## overlay template mask onto forms

In [200]:
oeeThresh = threshImg(oeeImg)

In [201]:
showImg(oeeThresh)

In [202]:
filledThreshold =0.7
circleSize = 4

omrRead =[]

for i, c in enumerate(sortedSpots):
    
    #maskBlue =np.full(img.shape, 255, dtype = "uint8") 
    mask =np.zeros(oeeThresh.shape, dtype = "uint8")
    cv2.circle(mask, tuple(c), circleSize, 255, -1)
    cv2.circle(maskBlue, tuple(c), circleSize, (255,0,0), -1)
    maskPixels = cv2.countNonZero(mask)
    mask = cv2.bitwise_and(oeeThresh, mask)
    pctFilled = cv2.countNonZero(mask)/maskPixels
    
    if pctFilled>filledThreshold:
        filled=True
    else:
        filled=False
    omrRead.append([i, c, pctFilled, filled])
    #showImg([mask,maskBlue])    
    #print(str(i))
    #showImg(mask)

## consolidate x and y vals

In [265]:
consMargin =10

xCoords = [xy[0] for xy in sortedSpots]
yCoords = [xy[1] for xy in sortedSpots]

uniqueX = []
for x in xCoords:
    if x not in uniqueX:
        uniqueX.append(x)
        
uniqueY = []
for y in yCoords:
    if y not in uniqueY:
        uniqueY.append(y)

xIndices = np.where(np.diff(np.array(sorted(uniqueX)))<consMargin)
yIndices = np.where(np.diff(np.array(sorted(uniqueY)))<consMargin)

repeatedX = np.array(sorted(uniqueX))[list(xIndices[0])]

repeatedY = np.array(sorted(uniqueY))[list(yIndices[0])]

#define replacement lists, i.e. first index is value to replace, second index is replacement values 
##(order doesnt matter as long as its consistent)

replacementY=[]
for y in repeatedY:    
    replacementY.append([i for i in uniqueY if y-consMargin<i<y+consMargin])

replacementX=[]

for x in repeatedX:    
    replacementX.append([i for i in uniqueX if x-5<i<x+5])

replacementSpots = copy.deepcopy(sortedSpots)

for i in replacementSpots:
    #print("i", i)
    for vals in replacementX:        
        if i[0]==vals[0]:
            print("match:", i[0], "to", vals[0], "changing to", vals[1])
            i[0]=vals[1]

for i in replacementSpots:
    #print("i", i)
    for vals in replacementY:        
        if i[1]==vals[0]:
            print("match:", i[1], "to", vals[0], "changing to", vals[1])
            i[1]=vals[1]

#check if coordinates have been consolidated
xCoords = [xy[0] for xy in replacementSpots]
yCoords = [xy[1] for xy in replacementSpots]

replacementUniqueX = []
for x in xCoords:
    if x not in replacementUniqueX:
        replacementUniqueX.append(x)
        
replacementUniqueY = []
for y in yCoords:
    if y not in replacementUniqueY:
        replacementUniqueY.append(y)

#which x values have been removed?
print("x vals consolidated:", set(uniqueX)- set(replacementUniqueX))

#which y values have been removed?
print("y vals consolidated:",set(uniqueY)- set(replacementUniqueY))

match: 60 to 60 changing to 63
match: 60 to 60 changing to 63
match: 60 to 60 changing to 63
match: 60 to 60 changing to 63
match: 60 to 60 changing to 63
match: 60 to 60 changing to 63
match: 60 to 60 changing to 63
match: 60 to 60 changing to 63
match: 60 to 60 changing to 63
match: 60 to 60 changing to 63
match: 107 to 107 changing to 109
match: 107 to 107 changing to 109
match: 107 to 107 changing to 109
match: 107 to 107 changing to 109
match: 107 to 107 changing to 109
match: 107 to 107 changing to 109
match: 107 to 107 changing to 109
match: 107 to 107 changing to 109
match: 107 to 107 changing to 109
match: 107 to 107 changing to 109
match: 107 to 107 changing to 109
match: 107 to 107 changing to 109
match: 154 to 154 changing to 156
match: 154 to 154 changing to 156
match: 154 to 154 changing to 156
match: 154 to 154 changing to 156
match: 154 to 154 changing to 156
match: 200 to 200 changing to 203
match: 200 to 200 changing to 203
match: 224 to 224 changing to 226
match: 224

## repeat consolidation (incase multiple spots within margin)

In [267]:
consMargin =10

xCoords = [xy[0] for xy in replacementSpots]
yCoords = [xy[1] for xy in replacementSpots]

uniqueX = []
for x in xCoords:
    if x not in uniqueX:
        uniqueX.append(x)
        
uniqueY = []
for y in yCoords:
    if y not in uniqueY:
        uniqueY.append(y)

xIndices = np.where(np.diff(np.array(sorted(uniqueX)))<consMargin)
yIndices = np.where(np.diff(np.array(sorted(uniqueY)))<consMargin)

repeatedX = np.array(sorted(uniqueX))[list(xIndices[0])]

repeatedY = np.array(sorted(uniqueY))[list(yIndices[0])]

#define replacement lists, i.e. first index is value to replace, second index is replacement values 
##(order doesnt matter as long as its consistent)

replacementY=[]
for y in repeatedY:    
    replacementY.append([i for i in uniqueY if y-consMargin<i<y+consMargin])

replacementX=[]

for x in repeatedX:    
    replacementX.append([i for i in uniqueX if x-5<i<x+5])

replacementSpots = copy.deepcopy(replacementSpots)

for i in replacementSpots:
    #print("i", i)
    for vals in replacementX:        
        if i[0]==vals[0]:
            print("match:", i[0], "to", vals[0], "changing to", vals[1])
            i[0]=vals[1]

for i in replacementSpots:
    #print("i", i)
    for vals in replacementY:        
        if i[1]==vals[0]:
            print("match:", i[1], "to", vals[0], "changing to", vals[1])
            i[1]=vals[1]

#check if coordinates have been consolidated
xCoords = [xy[0] for xy in replacementSpots]
yCoords = [xy[1] for xy in replacementSpots]

replacementUniqueX = []
for x in xCoords:
    if x not in replacementUniqueX:
        replacementUniqueX.append(x)
        
replacementUniqueY = []
for y in yCoords:
    if y not in replacementUniqueY:
        replacementUniqueY.append(y)

#which x values have been removed?
print("x vals consolidated:", set(uniqueX)- set(replacementUniqueX))

#which y values have been removed?
print("y vals consolidated:",set(uniqueY)- set(replacementUniqueY))

In [324]:
oeeImg = cv2.warpPerspective(imgResize, matrix, (maxWidth, maxHeight))

xTestYLine = [30] * 100
yTestXLine = [10] * 100

xTest = list(zip(list(np.array(sorted(replacementUniqueX), dtype='int64')), xTestYLine))

yTest = list(zip(yTestXLine, list(np.array(sorted(replacementUniqueY), dtype='int64'))))


for pt in xTest:    
    cv2.circle(oeeImg, pt, 2, (0,0,255))
    cv2.putText(oeeImg, str(pt[0]), pt, cv2.FONT_HERSHEY_PLAIN, 0.7, (0,0,0))
    cv2.line(oeeImg, (pt[0],0), (pt[0], 1000), (100,100,100), lineType=4)
    
for pt in yTest:    
    cv2.circle(oeeImg, pt, 2, (0,0,255))
    cv2.putText(oeeImg, str(pt[1]), pt, cv2.FONT_HERSHEY_PLAIN, 0.7, (0,0,0))
    cv2.line(oeeImg, (0,pt[1]), (1000, pt[1]), (100,100,100))
    
#cv2.imwrite("imgCoord.jpg", oeeImg)
showImg(oeeImg)

## play about with selection based of x, y coord range (see imgCoord.jpg for values)
## use this to populate templateSpot list and define qn/values

In [325]:
x1, x2 = [156,156]
y1, y2 = [321,387]

pickedSpots = [i for i in replacementSpots if x1<=i[0]<=x2 and y1<=i[1]<=y2]

oeeImg = cv2.warpPerspective(imgResize, matrix, (maxWidth, maxHeight))
for spot in pickedSpots:
    cv2.circle(oeeImg, tuple(spot), 4, (0,0,255))

showImg(oeeImg)